# Step 0: Defining Model Class

In [1]:
# Basics
import pandas as pd
import numpy as np

#Plotting
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
plt.style.use('ggplot')

# Warnings
import warnings
warnings.filterwarnings('ignore')

#Auto ARIMA
try:
    from pmdarima.arima import auto_arima
except:
    ! pip install pmdarima
    from pmdarima.arima import auto_arima

class Model(object):
    
    def __init__(self, test_size=48, predict_size=60):
        self.model_dictionary = {}
        self.test_size = test_size
        self.predict_size = predict_size
        
    def build_model(self):
        zip_codes = self.model_dictionary.keys()
        N = len(zip_codes)
        for ind, zip_code in enumerate(zip_codes):
            self.select_model(zip_code, trace=False)
            self.predict_in_sample(zip_code)
            self.predict(zip_code)
            print(f'Finished processing {ind+1} out of {N}, zip codes.')
    
    # Load data from data frame
    def load_data(self, df):
        self.make_datetime_index(df)
        for ind in df.index.values:
            row = df.loc[ind]
            zip_code = self.get_zip_code(row)
            row_dict = self.make_row_dict(row)
            self.model_dictionary[zip_code] = row_dict
    
    def make_datetime_index(self, df):
        string_index = df.columns.values[8:]
        self.datetime_index = pd.to_datetime(string_index)
            
    def get_zip_code(self, row):
        return row['RegionName']
            
    def make_row_dict(self, row):
        row_dict = row.iloc[1:8].to_dict()
        time_series = row.iloc[8:]
        df = self.make_time_series_df(time_series)
        df = df.fillna(df.bfill())
        row_dict['TimeSeries'] = df
        return row_dict
      
    def make_time_series_df(self, time_series):
        time_series.index = self.datetime_index
        df = pd.DataFrame(time_series)
        df.columns = ['MedianSales']
        return df
    
    def get_time_series(self, zip_code):
        return self.model_dictionary[zip_code]['TimeSeries']
    
    def get_city_name(self, zip_code):
        return self.model_dictionary[zip_code]['City']
    
    def get_state_abbreviation(self, zip_code):
        return self.model_dictionary[zip_code]['State']
    
    # Train Test Split    
    def make_train_test_split(self, zip_code):
        time_series_df = self.get_time_series(zip_code)
        train = time_series_df[:-self.test_size]
        test = time_series_df[-self.test_size:]
        return train, test
    
    # Plotting
    def make_time_series_plot(self, zip_code, plot_in_sample_prediction=True, prediction=True):
        train, test = self.make_train_test_split(zip_code)
        city_name = self.get_city_name(zip_code)
        state_abbreviation = self.get_state_abbreviation(zip_code)
        fig, ax = plt.subplots(figsize=(8, 5));
        ax.plot(train);
        ax.plot(test);
        legend_labels = ['Train', 'Test']
        try:
            in_sample_prediction = self.get_in_sample_prediction(zip_code)
            ax.plot(in_sample_prediction)
            legend_labels.append('Prediction')
        except:
            pass
        ax.legend(legend_labels);
        ax.set_title(f'Median Sale Price {city_name}, {state_abbreviation} {zip_code}')
        ax.set_xlabel('Date')
        ax.set_ylabel('Price')
        return fig
    
    def make_acf_plot(self, zip_code):
        city_name = self.get_city_name(zip_code)
        state_abbreviation = self.get_state_abbreviation(zip_code)
        train, test = self.make_train_test_split(zip_code)
        fig, ax = plt.subplots(figsize=(8,5));
        plot_acf(
            x=train, 
            ax=ax, 
            lags=100,
            title=f'Autocorrelation for {city_name}, {state_abbreviation} {zip_code}'
        );
        return fig
    
    def make_pacf_plot(self, zip_code):
        city_name = self.get_city_name(zip_code)
        state_abbreviation = self.get_state_abbreviation(zip_code)
        train, test = self.make_train_test_split(zip_code)
        fig, ax = plt.subplots(figsize=(8,5));
        plot_pacf(
            x=train, 
            ax=ax, 
            lags=100,
            title=f'Partial Autocorrelation for {city_name}, {state_abbreviation} {zip_code}'
        );
        return fig
    
    # Model Selection
    def select_model(self, zip_code, trace=True):
        y = self.get_time_series(zip_code)
        model = auto_arima(
            y = y,
            X=None,
            start_p=0,
            d=1,
            start_q=0,
            max_p=2,
            max_d=2,
            max_q=2,
            start_P=0,
            D=1,
            start_Q=0,
            max_P=2,
            max_D=2,
            max_Q=2,
            max_order=None,
            m=12,
            seasonal=True,
            stationary=False,
            information_criterion='oob',
            alpha=0.05,
            test='kpss',
            seasonal_test='OCSB',
            stepwise=True,            
            suppress_warnings=True,
            error_action='warn',
            trace=trace,
            out_of_sample_size= self.test_size,
            scoring='mse'
        )
        self.model_dictionary[zip_code]['BestModel'] = model
    
    def get_best_model(self, zip_code):
        try:
            best_model = self.model_dictionary[zip_code]['BestModel']
        except:
            self.select_model(zip_code)
            best_model = self.model_dictionary[zip_code]['BestModel']
        return best_model
    
    # Model Validation
    def predict_in_sample(self, zip_code):
        model = self.get_best_model(zip_code)
        time_series = self.get_time_series(zip_code)
        index = time_series.index
        in_sample_prediction = model.predict_in_sample()
        in_sample_prediction_df = pd.DataFrame(in_sample_prediction, index=index)
        in_sample_prediction_df.columns = ['MedianSales']
        self.model_dictionary[zip_code]['InSamplePrediction'] = in_sample_prediction_df
            
    def get_in_sample_prediction(self, zip_code):
        in_sample_prediction = self.model_dictionary[zip_code]['InSamplePrediction']
        return in_sample_prediction
    
    def plot_diagnostics(self, zip_code):
        best_model = self.get_best_model(zip_code)
        city = self.get_city_name(zip_code)
        state = self.get_state_abbreviation(zip_code)
        fig = best_model.plot_diagnostics(figsize=(16, 10));
        fig.suptitle(f'Diagnostics for {city}, {state} {zip_code}', fontsize=16)
        return fig
    
    # Predict future prices
    def predict(self, zip_code):
        model = self.get_best_model(zip_code)
        prediction = model.predict(n_periods=self.predict_size)
        self.model_dictionary[zip_code]['Prediction'] = prediction
        
    def get_prediction(self, zip_code):
        prediction = self.model_dictionary[zip_code]['Prediction']
        return prediction
    
    # Compute ROI
    def compute_roi(self, zip_code):
        initial_price = self.get_time_series(zip_code)['MedianSales'][-1]
        final_price = self.get_prediction(zip_code)[-1]
        roi = (final_price-initial_price)/initial_price
        self.model_dictionary[zip_code]['ROI'] = roi


# Step 1: Import the Data
Our main data set is stored in the `zillow_data.csv` spreadsheet. We load the CSV file as a data frame below.

In [2]:
df = pd.read_csv('../data/ZHVI.csv', index_col='RegionID')
print(df.shape)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/ZHVI.csv'

We will restrict our attention to the city of Baltimore, so we filter the data below.

In [ ]:
df = df.query("City == 'Baltimore' and State == 'MD'")
print(df.shape)
df.head()

# Step 2: Instantiate the Model and Load in the Data
Below we instantiate a Model object and load our selected data into the object.

In [ ]:
model = Model()
model.load_data(df)
model.build_model()

# Step 3: EDA and Visualization
Below we display a time series plot, ACF plot, and PACF plot for the zip code 21215. The class methods used below will work for any zip code in our model.

In [ ]:
model.make_time_series_plot(21215);

In [ ]:
model.make_acf_plot(21215);

In [ ]:
model.make_pacf_plot(21215);

# Step 4: ARIMA Modeling

# Step 5: Interpreting Results

In [ ]:
model.make_time_series_plot(21215);

In [ ]:
model.compute_roi(21215)

In [ ]:
model.plot_diagnostics(21215);

In [ ]:
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(model.model_dictionary, f)